In [1]:
# local imports
from src.transfer import replace_head, METRICS
from src.data import player_data
from src.util import display_progress

# computation / deep learning imports
from tensorflow import keras
import numpy as np

# os / filesystem imports
import pickle
import os

# visualization imports
from sklearn.metrics import confusion_matrix
import pandas as pd

# Model with Pre-Trained Base

In [2]:
# load model
model = keras.models.load_model('models/SSBML-Base-Model')

# replace head
model = replace_head(model)

In [3]:
model.summary()

Model: "SSBML-Transfer-Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
SSBML-Base-Model (Sequential (None, 512)               6537842   
_________________________________________________________________
Binary-Classifier (Sequentia (None, 1)                 83329     
Total params: 6,621,171
Trainable params: 82,817
Non-trainable params: 6,538,354
_________________________________________________________________


# The Players

In [4]:
!ls data/player

Blynde	CuckDaddy  gh0st  ixwonkr  Lie0x  TCBL


# The Player Data

Player Data (clips played by our chosen player) can be found in data/player/\<player name\>

Anonymous Data (clips not played by our chosen player) is taken from the large dataset data/character

In [5]:
# name of the player we want to train/test on
player_name = 'Blynde'

In [6]:
# set all filepath related variables

player_dir = os.path.join('data/player', player_name)
player_train_dir = os.path.join(player_dir, 'train')
player_test_dir = os.path.join(player_dir, 'test')
player_train_sample_size = len(os.listdir(player_train_dir))
player_test_sample_size = len(os.listdir(player_test_dir))

anonymous_dir = 'data/character'
anonymous_train_dir = os.path.join(anonymous_dir, 'train')
anonymous_test_dir = os.path.join(anonymous_dir, 'test')

print(f'Training Data Located at: \n\t- {player_train_dir} \n\t- {anonymous_train_dir} \n')
print(f'Testing Data Located at: \n\t- {player_test_dir} \n\t- {anonymous_test_dir} \n')
print(f'Player Training Data Sample size: \n\t- {player_train_sample_size} \n')
print(f'Player Testing Data Sample size: \n\t- {player_test_sample_size} \n')

Training Data Located at: 
	- data/player/Blynde/train 
	- data/character/train 

Testing Data Located at: 
	- data/player/Blynde/test 
	- data/character/test 

Player Training Data Sample size: 
	- 4286 

Player Testing Data Sample size: 
	- 499 



# Training

Train the model on a mix of the chosen player's clips,
and random anonymous clips from the Melee Public SLP Dataset

## Data Generation

In [7]:
# Adjustable Parameters
# =====================

DEFAULT_BATCH_SIZE = 16

# Anonymous Clips / Chosen Player's Clips
class_balance_ratio = 5

# Affects batch_size and steps_per_epoch
# Example: ratio of 2 would effectively 
#          double batch size and 
#          cut steps_per_epoch in half
tuning_ratio = 1

# =====================

# Calculate number of steps per epoch for train/test loops.
# One Epoch should iterate through our player's clips once, mixing them
# with random anonymous clips at our given class balance ratio 
train_steps = (
    (player_train_sample_size * (1 + class_balance_ratio))
    // (tuning_ratio * DEFAULT_BATCH_SIZE)
)
    
test_steps = (
    (player_test_sample_size * (1 + class_balance_ratio))
    // (tuning_ratio * DEFAULT_BATCH_SIZE)
)

# training data
training_data = player_data(
    player_train_dir,
    anonymous_train_dir,
    repeat = True,
    batch_size = DEFAULT_BATCH_SIZE * tuning_ratio,
    ratio = class_balance_ratio,
)

# testing data
testing_data = player_data(
    player_test_dir,
    anonymous_test_dir,
    repeat = True,
    batch_size = DEFAULT_BATCH_SIZE * tuning_ratio,
    ratio = class_balance_ratio,
)

## Training Loop

In [8]:
model.fit(
    training_data,
    epochs = 15,
    steps_per_epoch = train_steps,
    verbose = 1,
);

Epoch 1/15
1607/1607 [==============================] - 421s 260ms/step - loss: 0.0599 - accuracy: 0.8161 - precision: 0.4315 - recall: 0.1981 - specificity: 0.9356
Epoch 2/15
1607/1607 [==============================] - 419s 261ms/step - loss: 0.0306 - accuracy: 0.8654 - precision: 0.8240 - recall: 0.2441 - specificity: 0.9890
Epoch 3/15
1607/1607 [==============================] - 416s 259ms/step - loss: 0.0281 - accuracy: 0.8721 - precision: 0.8542 - recall: 0.3014 - specificity: 0.9888
Epoch 4/15
1607/1607 [==============================] - 418s 260ms/step - loss: 0.0266 - accuracy: 0.8796 - precision: 0.8707 - recall: 0.3142 - specificity: 0.9891
Epoch 5/15
1607/1607 [==============================] - 419s 261ms/step - loss: 0.0265 - accuracy: 0.8769 - precision: 0.8648 - recall: 0.3147 - specificity: 0.9901
Epoch 6/15
1607/1607 [==============================] - 420s 261ms/step - loss: 0.0254 - accuracy: 0.8838 - precision: 0.8739 - recall: 0.3428 - specificity: 0.9903
Epoch 7/15

In [9]:
# Adjustable Parameters
# =====================

DEFAULT_BATCH_SIZE = 16

# Anonymous Clips / Chosen Player's Clips
class_balance_ratio = 3

# Affects batch_size and steps_per_epoch
# Example: ratio of 2 would effectively 
#          double batch size and 
#          cut steps_per_epoch in half
tuning_ratio = 1

# =====================

# Calculate number of steps per epoch for train/test loops.
# One Epoch should iterate through our player's clips once, mixing them
# with random anonymous clips at our given class balance ratio 
train_steps = (
    (player_train_sample_size * (1 + class_balance_ratio))
    // (tuning_ratio * DEFAULT_BATCH_SIZE)
)
    
test_steps = (
    (player_test_sample_size * (1 + class_balance_ratio))
    // (tuning_ratio * DEFAULT_BATCH_SIZE)
)

# training data
training_data = player_data(
    player_train_dir,
    anonymous_train_dir,
    repeat = True,
    batch_size = DEFAULT_BATCH_SIZE * tuning_ratio,
    ratio = class_balance_ratio,
)

# testing data
testing_data = player_data(
    player_test_dir,
    anonymous_test_dir,
    repeat = True,
    batch_size = DEFAULT_BATCH_SIZE * tuning_ratio,
    ratio = class_balance_ratio,
)

In [10]:
model.fit(
    training_data,
    epochs = 5,
    steps_per_epoch = train_steps,
    verbose = 1,
);

Epoch 1/5
1071/1071 [==============================] - 279s 261ms/step - loss: 0.0281 - accuracy: 0.8662 - precision: 0.9026 - recall: 0.5189 - specificity: 0.9814
Epoch 2/5
1071/1071 [==============================] - 279s 260ms/step - loss: 0.0279 - accuracy: 0.8688 - precision: 0.9036 - recall: 0.5262 - specificity: 0.9815
Epoch 3/5
1071/1071 [==============================] - 282s 263ms/step - loss: 0.0271 - accuracy: 0.8742 - precision: 0.9081 - recall: 0.5375 - specificity: 0.9825
Epoch 4/5
1071/1071 [==============================] - 279s 260ms/step - loss: 0.0279 - accuracy: 0.8690 - precision: 0.9025 - recall: 0.5315 - specificity: 0.9810
Epoch 5/5
1071/1071 [==============================] - 277s 259ms/step - loss: 0.0274 - accuracy: 0.8714 - precision: 0.9006 - recall: 0.5425 - specificity: 0.9802


In [11]:
# Adjustable Parameters
# =====================

DEFAULT_BATCH_SIZE = 16

# Anonymous Clips / Chosen Player's Clips
class_balance_ratio = 2

# Affects batch_size and steps_per_epoch
# Example: ratio of 2 would effectively 
#          double batch size and 
#          cut steps_per_epoch in half
tuning_ratio = 1

# =====================

# Calculate number of steps per epoch for train/test loops.
# One Epoch should iterate through our player's clips once, mixing them
# with random anonymous clips at our given class balance ratio 
train_steps = (
    (player_train_sample_size * (1 + class_balance_ratio))
    // (tuning_ratio * DEFAULT_BATCH_SIZE)
)
    
test_steps = (
    (player_test_sample_size * (1 + class_balance_ratio))
    // (tuning_ratio * DEFAULT_BATCH_SIZE)
)

# training data
training_data = player_data(
    player_train_dir,
    anonymous_train_dir,
    repeat = True,
    batch_size = DEFAULT_BATCH_SIZE * tuning_ratio,
    ratio = class_balance_ratio,
)

# testing data
testing_data = player_data(
    player_test_dir,
    anonymous_test_dir,
    repeat = True,
    batch_size = DEFAULT_BATCH_SIZE * tuning_ratio,
    ratio = class_balance_ratio,
)

In [12]:
model.fit(
    training_data,
    epochs = 5,
    steps_per_epoch = train_steps,
    verbose = 1,
);

Epoch 1/5
803/803 [==============================] - 210s 261ms/step - loss: 0.0314 - accuracy: 0.8517 - precision: 0.9121 - recall: 0.6071 - specificity: 0.9714
Epoch 2/5
803/803 [==============================] - 210s 261ms/step - loss: 0.0298 - accuracy: 0.8617 - precision: 0.9171 - recall: 0.6333 - specificity: 0.9723
Epoch 3/5
803/803 [==============================] - 210s 261ms/step - loss: 0.0303 - accuracy: 0.8589 - precision: 0.9118 - recall: 0.6295 - specificity: 0.9704
Epoch 4/5
803/803 [==============================] - 209s 260ms/step - loss: 0.0305 - accuracy: 0.8583 - precision: 0.9095 - recall: 0.6405 - specificity: 0.9679
Epoch 5/5
803/803 [==============================] - 212s 265ms/step - loss: 0.0300 - accuracy: 0.8587 - precision: 0.9082 - recall: 0.6375 - specificity: 0.9681


In [13]:
# Adjustable Parameters
# =====================

DEFAULT_BATCH_SIZE = 16

# Anonymous Clips / Chosen Player's Clips
class_balance_ratio = 1

# Affects batch_size and steps_per_epoch
# Example: ratio of 2 would effectively 
#          double batch size and 
#          cut steps_per_epoch in half
tuning_ratio = 4

# =====================

# Calculate number of steps per epoch for train/test loops.
# One Epoch should iterate through our player's clips once, mixing them
# with random anonymous clips at our given class balance ratio 
train_steps = (
    (player_train_sample_size * (1 + class_balance_ratio))
    // (tuning_ratio * DEFAULT_BATCH_SIZE)
)
    
test_steps = (
    (player_test_sample_size * (1 + class_balance_ratio))
    // (tuning_ratio * DEFAULT_BATCH_SIZE)
)

# training data
training_data = player_data(
    player_train_dir,
    anonymous_train_dir,
    repeat = True,
    batch_size = DEFAULT_BATCH_SIZE * tuning_ratio,
    ratio = class_balance_ratio,
)

# testing data
testing_data = player_data(
    player_test_dir,
    anonymous_test_dir,
    repeat = True,
    batch_size = DEFAULT_BATCH_SIZE * tuning_ratio,
    ratio = class_balance_ratio,
)

In [14]:
model.fit(
    training_data,
    epochs = 1,
    steps_per_epoch = train_steps,
    verbose = 1,
);

133/133 [==============================] - 93s 700ms/step - loss: 0.0308 - accuracy: 0.8600 - precision: 0.9387 - recall: 0.7723 - specificity: 0.9489


# Testing

Test the model on one pass of the given player's clips, 
at the given class balance ratio

In [20]:
# Adjustable Parameters
# =====================

DEFAULT_BATCH_SIZE = 16

# Ratio of Anonymous clips : Chosen Player's clips
class_balance_ratio = 1

# =====================

data_test_one_round = player_data(
    player_test_dir,
    anonymous_test_dir,
    batch_size = DEFAULT_BATCH_SIZE,
    repeat = False,
    ratio = class_balance_ratio,
)

score = model.evaluate(data_test_one_round, verbose=1)
print('\nTest score:', round(score[0], 3))
print(f'- accuracy: {round(score[1]*100)}%')
print(f'- precision: {round(score[2]*100)}%')
print(f'- recall: {round(score[3]*100)}%')
print(f'- specificity: {round(score[4]*100)}%')

63/63 [==============================] - 13s 205ms/step - loss: 0.0292 - accuracy: 0.8919 - precision: 0.9196 - recall: 0.8545 - specificity: 0.9279

Test score: 0.029
- accuracy: 89%
- precision: 92%
- recall: 85%
- specificity: 93%


## Confusion Matrix

In [19]:
# Adjustable Parameters
# =====================

DEFAULT_BATCH_SIZE = 16

# Ratio of Anonymous clips : Chosen Player's clips
class_balance_ratio = 1

# =====================

# define data generation
data_conf = player_data(
    player_test_dir,
    anonymous_test_dir,
    batch_size = DEFAULT_BATCH_SIZE,
    ratio = class_balance_ratio,
)

# predict over test data
batch_preds = []
batch_labels = []
i = 0
N = player_test_sample_size
for xi, yi in data_conf:
    batch_preds.append(model.predict(xi).round().astype(int))
    batch_labels.append(yi.astype(int))
    
    # progess bar
    i = int(i + np.sum(yi))
    display_progress(i, N)
display_progress(N, N)
print('\n')

pred = np.concatenate(batch_preds)
labels = np.concatenate(batch_labels)

# create confusion matrix
# reverse so true positive is top left, true negative is bottom right
conf_matrix = confusion_matrix(labels, pred, normalize='all')[::-1, ::-1] 
conf_matrix = np.around(conf_matrix, 3)
conf_df = pd.DataFrame(
    conf_matrix, 
    index = ['present', 'not present'], 
    columns = ['detected', 'not detected']
)

conf_df

[####################] 499 of 499 - 100.0% 



,detected,not detected
present,0.391,0.068
not present,0.041,0.500
